In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import VBox
from IPython.display import display, clear_output
import os
import numpy as np

# CONFIGURATION: PATHS TO OFFLINE ANALYSIS FILES
COMPOSITE_MODEL_CSV = 'activations/quantitative_analysis_results_composite_new_loss_layer3.csv'
SINGLE_MODEL_CSV = 'activations/quantitative_analysis_results.csv'
if not os.path.exists(COMPOSITE_MODEL_CSV) or not os.path.exists(SINGLE_MODEL_CSV):
    print("Error: One or both CSV files were not found. Please ensure the paths are correct.")
    print(f"Checked for '{os.path.abspath(COMPOSITE_MODEL_CSV)}' and '{os.path.abspath(SINGLE_MODEL_CSV)}'.")
else:
    df_composite = pd.read_csv(COMPOSITE_MODEL_CSV)
    df_single = pd.read_csv(SINGLE_MODEL_CSV)
    
    df_composite['Model Type'] = 'Composite'
    df_single['Model Type'] = 'Single MLP'
    
    df_all = pd.concat([df_composite, df_single], ignore_index=True)
    print("Successfully loaded and combined both analysis files.")

    # Setup Interactive Widgets
    available_layers = sorted(df_all['Layer'].unique().tolist())
    default_layer = available_layers[2] if available_layers else None
    
    layer_selector = widgets.Dropdown(options=available_layers, value=default_layer, description='Layer:')
    output_area = widgets.Output()

    def handle_summary_update(change):
        with output_area:
            clear_output(wait=True)
            selected_layer = layer_selector.value

            df_layer = df_all[df_all['Layer'] == selected_layer]

            final_data = []
            final_index = []

            # Add 'Comparison (Avg of Parts)' summary rows
            df_semantic = df_layer[df_layer['Grouping'] == 'Semantic Parts']
            if not df_semantic.empty:
                avg_of_parts_summary = df_semantic.groupby('Model Type').mean(numeric_only=True)
                
                if 'Composite' in avg_of_parts_summary.index:
                    final_index.append(('Avg of Parts', 'Composite Model'))
                    final_data.append(avg_of_parts_summary.loc['Composite'])
                if 'Single MLP' in avg_of_parts_summary.index:
                    final_index.append(('Avg of Parts', 'Single MLP'))
                    final_data.append(avg_of_parts_summary.loc['Single MLP'])

            # Add Detailed Per-Part Breakdowns
            if not df_semantic.empty:
                for model_type, group_df in df_semantic.groupby('Model Type'):
                    part_summary = group_df.groupby('Group Name').mean(numeric_only=True)
                    analysis_name = f"{model_type}"
                    for name, row in part_summary.iterrows():
                        final_index.append((analysis_name, name))
                        final_data.append(row)

            if not final_data:
                print(f"No data found for layer '{selected_layer}'."); return

            final_df = pd.DataFrame(final_data, index=pd.MultiIndex.from_tuples(final_index, names=["Analysis", "Group"]))
            
            # Rename some collumns (Maybe should have been done beforehand)
            final_df.rename(columns={
                'Sparsity (%)': 'Active Neurons (%)',
                'Global Mean Act': 'Mean Activation',
                'Global Variance': 'Mean Variance',
                'Disentanglement (Avg Jaccard)': 'Avg Jaccard'
            }, inplace=True)
            
            desired_cols = [
                'Active Neurons (%)', 'Mean Activation', 'Mean Variance', 'Avg Jaccard', 'Avg Top-5 Mean Act', "Avg Top-5 Variance", "Specialized Neurons"
            ]
            final_df = final_df.reindex(columns=[col for col in desired_cols if col in final_df.columns])

            style_cols_green = ['Mean Activation']
            style_cols_red = ['Active Neurons (%)', 'Mean Variance', 'Avg Jaccard']
            
            styled_df = final_df.style \
                .background_gradient(cmap='viridis', subset=[c for c in style_cols_green if c in final_df], axis=0) \
                .background_gradient(cmap='viridis_r', subset=[c for c in style_cols_red if c in final_df], axis=0) \
                .format({
                    'Active Neurons (%)': "{:.0f}",  # Round to integer
                    'Mean Activation': "{:.2f}",
                    'Mean Variance': "{:.2f}",
                    'Avg Jaccard': "{:.2f}"
                }, na_rep="-") \
                .set_caption(f"--- Comprehensive Analysis for Layer: '{selected_layer}' ---")

            display(styled_df)
            
    layer_selector.observe(handle_summary_update, names='value')
    display(VBox([layer_selector, output_area]))
    handle_summary_update(None)

Successfully loaded and combined both analysis files.


In [2]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import VBox
from IPython.display import display, clear_output
import os
import numpy as np

# CONFIGURATION of Analysis files
COMPOSITE_MODEL_CSV = 'activations/quantitative_analysis_results_composite_new_loss_layer3.csv'
COMPOSITE_MODEL_ZEROD_CSV = 'activations/quantitative_analysis_results_composite_new_loss_zerod.csv'
SINGLE_MODEL_CSV = 'activations/quantitative_analysis_results.csv'


# Load and Unify the DataFrames
df_all = None
if not os.path.exists(COMPOSITE_MODEL_CSV) or not os.path.exists(SINGLE_MODEL_CSV):
    print("Error: One or both CSV files were not found. Please ensure the paths are correct.")
    print(f"Checked for '{COMPOSITE_MODEL_CSV}' and '{SINGLE_MODEL_CSV}'.")
else:
    df_composite = pd.read_csv(COMPOSITE_MODEL_CSV)
    df_single = pd.read_csv(SINGLE_MODEL_CSV)
    
    df_composite['Model Type'] = 'Composite'
    df_single['Model Type'] = 'Single MLP'
    
    df_all = pd.concat([df_composite, df_single], ignore_index=True)
    print("Successfully loaded and combined both analysis files.")

    # Setup Interactive Widgets
    print(df_all['Layer'].unique())
    available_layers = sorted(df_all['Layer'].unique().tolist())
    default_layer = available_layers[len(available_layers) // 2] if available_layers else None
    
    layer_selector = widgets.Dropdown(options=available_layers, value=default_layer, description='Layer:')
    output_area = widgets.Output()

    def handle_summary_update(change):
        with output_area:
            clear_output(wait=True)
            if df_all is None:
                print("Data not loaded due to file path error."); return
            
            selected_layer = layer_selector.value
            df_layer = df_all[df_all['Layer'] == selected_layer]

            # Only Keep the Summary Rows
            final_data = []
            final_index = []

            # Add 'Comparison (Avg of Parts)' summary rows
            df_semantic = df_layer[df_layer['Grouping'] == 'Semantic Parts']
            if not df_semantic.empty:
                avg_of_parts_summary = df_semantic.groupby('Model Type').mean(numeric_only=True)
                
                if 'Composite' in avg_of_parts_summary.index:
                    final_index.append(('Avg of Parts', 'Composite Model'))
                    final_data.append(avg_of_parts_summary.loc['Composite'])
                if 'Single MLP' in avg_of_parts_summary.index:
                    final_index.append(('Avg of Parts', 'Single MLP'))
                    final_data.append(avg_of_parts_summary.loc['Single MLP'])


            if not final_data:
                print(f"No summary data found for layer '{selected_layer}'."); return

            final_df = pd.DataFrame(final_data, index=pd.MultiIndex.from_tuples(final_index, names=["Analysis", "Group"]))
            
            final_df.rename(columns={
                'Sparsity (%)': 'Active Neurons (%)',
                'Global Mean Act': 'Mean Activation',
                'Global Variance': 'Mean Variance',
                'Disentanglement (Avg Jaccard)': 'Avg Jaccard'
            }, inplace=True)
            
            desired_cols = [
                'Active Neurons (%)', 'Mean Activation', 'Mean Variance', 'Avg Jaccard'
            ]
            final_df = final_df.reindex(columns=[col for col in desired_cols if col in final_df.columns])

            style_cols_green = []
            style_cols_red = ['Active Neurons (%)', 'Mean Variance', 'Avg Jaccard', 'Mean Activation']
            
            styled_df = final_df.style \
                .background_gradient(cmap='viridis', subset=[c for c in style_cols_green if c in final_df], axis=0) \
                .background_gradient(cmap='viridis_r', subset=[c for c in style_cols_red if c in final_df], axis=0) \
                .format({
                    'Active Neurons (%)': "{:.0f}",  # Round to integer
                    'Mean Activation': "{:.2f}",
                    'Mean Variance': "{:.2f}",
                    'Avg Jaccard': "{:.2f}"
                }, na_rep="-") \
                .set_caption(f"--- Average Analysis for Layer: '{selected_layer}' ---")

            display(styled_df)
            
    if df_all is not None:
        layer_selector.observe(handle_summary_update, names='value')
        display(VBox([layer_selector, output_area]))
        handle_summary_update(None)

Successfully loaded and combined both analysis files.
['hidden_layer_0' 'hidden_layer_1' 'hidden_layer_2'
 'hidden_layer_3_readout']


In [ ]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import VBox
from IPython.display import display, clear_output
import os
import numpy as np

# CONFIGURATION of Analysis files
COMPOSITE_MODEL_CSV = 'activations/quantitative_analysis_results_composite_new_loss.csv'
COMPOSITE_MODEL_ZEROD_CSV = 'activations/quantitative_analysis_results_composite_new_loss_zerod.csv'
SINGLE_MODEL_CSV = 'activations/quantitative_analysis_results_single.csv'


# Load and Unify the DataFrames
df_all = None
# Check if all files exist
if not all(os.path.exists(f) for f in [COMPOSITE_MODEL_CSV, COMPOSITE_MODEL_ZEROD_CSV, SINGLE_MODEL_CSV]):
    print("Error: One or more CSV files were not found. Please ensure the paths are correct.")
else:
    df_composite = pd.read_csv(COMPOSITE_MODEL_CSV)
    df_composite_zerod = pd.read_csv(COMPOSITE_MODEL_ZEROD_CSV)
    df_single = pd.read_csv(SINGLE_MODEL_CSV)
    
    df_composite['Model Type'] = 'Composite'
    df_composite_zerod['Model Type'] = 'Composite (Zeroed)'
    df_single['Model Type'] = 'Single MLP'
    
    df_all = pd.concat([df_composite, df_composite_zerod, df_single], ignore_index=True)
    print("Successfully loaded and combined all analysis files.")

    # Setup Interactive Widgets
    available_layers = sorted(df_all['Layer'].unique().tolist())
    default_layer = available_layers[len(available_layers) // 2] if available_layers else None
    
    layer_selector = widgets.Dropdown(options=available_layers, value=default_layer, description='Layer:')
    output_area = widgets.Output()

    def handle_summary_update(change):
        with output_area:
            clear_output(wait=True)
            if df_all is None:
                print("Data not loaded due to file path error."); return
            
            selected_layer = layer_selector.value
            df_layer = df_all[df_all['Layer'] == selected_layer]

            # Only Keep the Summary Rows
            final_data = []
            final_index = []

            # Add 'Comparison (Avg of Parts)' summary rows
            df_semantic = df_layer[df_layer['Grouping'] == 'Semantic Parts']
            if not df_semantic.empty:
                avg_of_parts_summary = df_semantic.groupby('Model Type').mean(numeric_only=True)
                
                # 1. Composite (Standard)
                if 'Composite' in avg_of_parts_summary.index:
                    final_index.append(('Avg of Parts', 'Composite Model'))
                    final_data.append(avg_of_parts_summary.loc['Composite'])
                
                # 2. Composite (Zeroed / Inference Mode)
                if 'Composite (Zeroed)' in avg_of_parts_summary.index:
                    final_index.append(('Avg of Parts', 'Composite (Zeroed)'))
                    final_data.append(avg_of_parts_summary.loc['Composite (Zeroed)'])

                # 3. Single MLP
                if 'Single MLP' in avg_of_parts_summary.index:
                    final_index.append(('Avg of Parts', 'Single MLP'))
                    final_data.append(avg_of_parts_summary.loc['Single MLP'])


            if not final_data:
                print(f"No summary data found for layer '{selected_layer}'."); return

            final_df = pd.DataFrame(final_data, index=pd.MultiIndex.from_tuples(final_index, names=["Analysis", "Group"]))
            
            final_df.rename(columns={
                'Sparsity (%)': 'Active Neurons (%)',
                'Global Mean Act': 'Mean Activation',
                'Global Variance': 'Mean Variance',
                'Disentanglement (Avg Jaccard)': 'Avg Jaccard'
            }, inplace=True)
            
            desired_cols = [
                'Active Neurons (%)', 'Mean Activation', 'Mean Variance', 'Avg Jaccard'
            ]
            final_df = final_df.reindex(columns=[col for col in desired_cols if col in final_df.columns])

            style_cols_green = []
            style_cols_red = ['Active Neurons (%)', 'Mean Variance', 'Avg Jaccard', 'Mean Activation']
            
            styled_df = final_df.style \
                .background_gradient(cmap='viridis', subset=[c for c in style_cols_green if c in final_df], axis=0) \
                .background_gradient(cmap='viridis_r', subset=[c for c in style_cols_red if c in final_df], axis=0) \
                .format({
                    'Active Neurons (%)': "{:.0f}",  # Round to integer
                    'Mean Activation': "{:.2f}",
                    'Mean Variance': "{:.2f}",
                    'Avg Jaccard': "{:.2f}"
                }, na_rep="-") \
                .set_caption(f"--- Average Analysis for Layer: '{selected_layer}' ---")

            display(styled_df)
            
    if df_all is not None:
        layer_selector.observe(handle_summary_update, names='value')
        display(VBox([layer_selector, output_area]))
        handle_summary_update, None

Successfully loaded and combined all analysis files.
